In [1]:
import copy
import sub.plot as pl
import sub.functions as sb
from global_variables import gparam

gl = gparam()

In [2]:
condition = {
    # TF current
    'cur_tf':{'tf': +50.0e+3, 'turn': 16},
    
    # initial plasma profile
    'cur_ip':{'ip':+100.0e+3, 'r0':0.65, 'z0':0.0, 'radius':0.3},
       
    # PF currents
    'cur_pf':{'hcult16':0.0,'pf17t12':-1.0e+3, 'pf26t36':-1.0e+3,'pf4_1ab3_cc2':0.0,'pf35_2':0.0, },
        
    # number of coefficients
    'num_dpr':1, # dp/df
    'num_di2':1, # di2/df
    
    # calculate flux (r, z): result is set to 'fl_val'.
    'fl_pos':{'flc8':(0.1985, 0.450), 'f_im':(0.1985, 0.0), 'flc17':(0.1985, -0.450),
              'fls1':(1.374, 0.450), 'fls5':(1.374, 0.0), 'fls9':(1.374, -0.481)},
    
    'resolution': gl.get_dmat_coarse(),
    }

In [3]:
# 計算条件のリスト作成
pos = [0.5, 0.55, 0.6, 0.65, 0.7]
list_cond=[]
for e in pos:
    f = copy.deepcopy(condition)
    f['cur_ip']['r0'] = e
    list_cond.append(f)

In [4]:
eqs = [sb.calc_equilibrium(e, verbose=0) for e in list_cond]

In [5]:
[e['cal_result'] for e in eqs]

[1, 1, 1, 1, 1]

In [6]:
[pl.plot_fundamental(e) for e in eqs]
# jt = 2pi*r*(dp/df)+(u0/4pi)(1/r)(di^2/df)
# x = (f-f_axis)/(f_surf-f_axis) 
# should be: p > 0
#
# case: ip > 0
# should be: dp/df>0, di^2/df>0? for positive current
#
# case: ip < 0
# should be: dp/df<0, di^2/df<0? for negative current
#
# How to check these plot
# First, check which side of the plot is the magnetic axis or boundary.
# The boundary is where p[x] = 0, and make sure p[x] > 0.
# If p[x] < 0, please try to increase its vertical field.
# Next, check the positive/negative of (dp/df) and (di^2/df).
# These represent the directions of toroidal currents derived from pressure and poloidal currents, respectively.
# Finally, check the absolute amounts of 2pi(dp/df) and (u0/4pi)(di^2/df).
# Each of these represents a contribution to toroidal current.

[None, None, None, None, None]

In [ ]:
[e['r_ax'] for e in eqs]

In [ ]:
pl.plot_val(cond, 'safety_factor_norm')

In [ ]:
# Plasma current profile
pl.d_contour(cond['jt'])

In [ ]:
# calculate jt
import numpy as np
# r位置のマトリックスを作成
g = sb.dm_array(copy.deepcopy(condition['resolution']))
rm = [np.linspace(g.rmin, g.rmax, g.nr)]*g.nz
rm = np.array(rm)+10**(-7)
jt = 0
jt += 2.0*np.pi*rm*cond['diff_pre']['matrix'] # 圧力由来
jt += 10**(-7)/rm*cond['diff_i2']['matrix'] # pol cur由来
pl.contour(jt, g.rmin, g.zmin, g.dr, g.dz)
print('total current: ', jt.sum()*g.dr*g.dz)

In [ ]:
# pressure
pl.d_contour(cond['pressure'])

In [ ]:
# poloidal current
pl.d_contour(cond['pol_current'])

In [ ]:
pl.d_heatmap(cond['safety_factor'])

In [ ]:
# vessel 
pl.d_heatmap(cond['vessel'])

In [ ]:
# domain
pl.d_heatmap(cond['domain'])